In [92]:
import pandas as pd

In [93]:
estacion = "SURESTE3"

d20_21 = pd.read_excel('DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx', sheet_name=estacion)
d22_23 = pd.read_excel('DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx', sheet_name=estacion)
df = pd.concat([d20_21,d22_23])

In [94]:
df['date'] = pd.to_datetime(df['date'])
df["deltaDate"] = (df["date"] - df.iloc[0,0]).apply(lambda x: x.value/3.6e+12)
df.head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,deltaDate
0,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,37.0,18.0,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.0
1,2020-01-01 01:00:00,1.88,2.0,5.2,7.2,25.0,37.0,28.0,734.2,0.01,93.0,NaN,0.037,11.68,1.7,98.0,1.0
2,2020-01-01 02:00:00,1.86,2.6,5.0,7.6,22.0,55.0,28.0,733.8,0.02,93.0,NaN,0.037,11.68,1.6,133.0,2.0
3,2020-01-01 03:00:00,1.78,3.1,5.9,8.9,21.0,35.0,11.0,733.0,0.00,93.0,NaN,0.037,11.70,1.5,198.0,3.0
4,2020-01-01 04:00:00,1.76,2.0,4.5,6.5,22.0,27.0,NaN,732.8,0.02,93.0,NaN,0.037,11.80,0.7,240.0,4.0


In [95]:
print(df.shape)
df.isna().sum()

(31790, 17)


date            0
CO           2084
NO           1467
NO2          1263
NOX          1240
O3           3353
PM10         1565
PM2.5        3690
PRS           967
RAINF         911
RH           1198
SO2          2886
SR            141
TOUT          915
WSR           939
WDR           944
deltaDate       0
dtype: int64

In [96]:
from sklearn.impute import KNNImputer as imputer
k = imputer(n_neighbors=4)
datos = k.fit_transform((df.iloc[:, 1:]).to_numpy())

In [97]:
limpios = pd.DataFrame(datos, columns=df.columns[1:])
limpios["date"] = list(df["date"])
limpios.to_csv(f"datosCompletos{estacion}.csv")

In [98]:
intervalos={
    "SO2":[0.008, 0.11, 0.165, 0.22],
    "NO2":[8.75, 11, 13.3, 15.5],
    "etiqueta": ["bueno", "aceptable", "mala", "muy mala"]
}
def change(val, col):
    if val< intervalos[col][0]:
        return "bueno"
    if val< intervalos[col][1]:
        return "aceptable"
    if val< intervalos[col][2]:
        return "mala"
    if val< intervalos[col][3]:
        return "muy mala"
    return "Extremadamente mala"

In [99]:
datos = limpios[["date", "deltaDate","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR", "PM2.5"]]
cols_ppm = ['NO','NO2','NOX','SO2']
for col in cols_ppm:
    datos[col+'PPM'] = datos[col]*0.001
datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))
datos.to_csv(f"datos{estacion}.csv")

C:\Users\engel\AppData\Local\Temp\ipykernel_3512\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos[col+'PPM'] = datos[col]*0.001
C:\Users\engel\AppData\Local\Temp\ipykernel_3512\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos[col+'PPM'] = datos[col]*0.001
C:\Users\engel\AppData\Local\Temp\ipykernel_3512\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [100]:
datos.head()

,date,deltaDate,NO,NO2,NOX,SO2,TOUT,RH,SR,PM2.5,NOPPM,NO2PPM,NOXPPM,SO2PPM,SO2_discret
0,2020-01-01 00:00:00,0.0,4.3,6.375,10.625,0.850,12.2675,92.75,0.000,18.00,0.0043,0.006375,0.010625,0.000850,bueno
1,2020-01-01 01:00:00,1.0,2.0,5.200,7.200,1.775,11.6800,93.00,0.037,28.00,0.0020,0.005200,0.007200,0.001775,bueno
2,2020-01-01 02:00:00,2.0,2.6,5.000,7.600,0.750,11.6800,93.00,0.037,28.00,0.0026,0.005000,0.007600,0.000750,bueno
3,2020-01-01 03:00:00,3.0,3.1,5.900,8.900,0.725,11.7000,93.00,0.037,11.00,0.0031,0.005900,0.008900,0.000725,bueno
4,2020-01-01 04:00:00,4.0,2.0,4.500,6.500,0.775,11.8000,93.00,0.037,10.25,0.0020,0.004500,0.006500,0.000775,bueno


In [101]:
datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))

#limpios["NO2_discret"] = limpios["NO2"].apply(lambda x: change(x, "NO2"))
#limpios["SO2_discret"] = (limpios["SO2"]*0.001).apply(lambda x: change(x, "SO2"))

C:\Users\engel\AppData\Local\Temp\ipykernel_3512\3441339858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))


In [102]:
#limpios["SO2_pm"] = limpios["SO2"]*0.001

In [103]:
#limpios.head()

In [104]:
datos = limpios[["date", "deltaDate","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR", "NO2_discret",	"SO2_discret",	"SO2_pm"]]

KeyError: "['NO2_discret', 'SO2_discret', 'SO2_pm'] not in index"

In [ ]:
datos.head()

,date,deltaDate,NO,NO2,NOX,SO2,TOUT,RH,SR,NO2_discret,SO2_discret,SO2_pm
0,2020-01-01 00:00:00,0.0,4.3,6.375,10.625,0.850,12.2675,92.75,0.000,bueno,bueno,0.000850
1,2020-01-01 01:00:00,1.0,2.0,5.200,7.200,1.775,11.6800,93.00,0.037,bueno,bueno,0.001775
2,2020-01-01 02:00:00,2.0,2.6,5.000,7.600,0.750,11.6800,93.00,0.037,bueno,bueno,0.000750
3,2020-01-01 03:00:00,3.0,3.1,5.900,8.900,0.725,11.7000,93.00,0.037,bueno,bueno,0.000725
4,2020-01-01 04:00:00,4.0,2.0,4.500,6.500,0.775,11.8000,93.00,0.037,bueno,bueno,0.000775


In [ ]:
#datos.to_csv("limpios.csv")